# Open and Shut Case: The Optimal Strategy for Shut the Box

David Castner

Shut the Box is a simple dice roll game where the player has a set of tiles numbered 1 to 9. The player rolls two dice and then flips down tiles that sum to the dice roll. The player wins if all tiles are flipped down. The player loses if there are no valid moves left. The player can also choose to stop rolling the dice and flip down the remaining tiles. The player's score is the sum of the tiles that were flipped down. The goal of this project is to find the optimal strategy for Shut the Box.

Example: The player rolls a 6 and a 3. The player can flip down the 6 and the 3, or the 6 and the 2 and the 1, or the 5 and the 4. The player can also choose to stop rolling the dice and flip down the 7, 8, and 9. If the player chooses to stop rolling the dice, the player's score is 24. If the player chooses to roll again, the player's score is 11.

## Hypothesis

I hypothesize that the optimal strategy for Shut the Box is to flip down the least amount of tiles and then choose to flip the highest possible tiles each turn. This would leave lower tiles numbers allowing for more possible combinations of dice rolls later in the game. For example, if the first roll was a 9, then flipping over the 9 would be the best move. If another 9 is rolled, then flipping the 1 and 8 next would be the best move.

## Finding the Optimal Strategy

> Definitions: A **board state** is a unique combination of tiles that are up or down.

Finding the optimal strategy can be done using information theory. Each possible board state and roll can be calculated since the amount of combinations is small for modern computing. If a tile can either be flipped or not be flipped, it is equivalent to a single bit of information. With 9 different tiles, there are 2^9 possible board states. With 2 dice, there are 6^2 possible rolls. This means there are 2^9 * 6^2 = 18432 possible board states and rolls. This is a small enough number to calculate all possible board states and rolls.

Calculating the expected score of a board state can be calculated by summing the probability of each roll multiplied by the maximum of each possible resulting board state's expected score. Below is the formula for that calculation: $$S(b) = \sum_{i=d_n}^{d_m} P(i) \cdot F(\{S(b') | b' \in B(i, b)\})$$

Where $S(b)$ is the expected score of board state $b$, $P(i)$ is the probability of roll $i$, $d_n$ is the minimum roll, $d_m$ is the maximum roll, $B(i, b)$ is the set of all possible board states resulting from roll $i$ and board state $b$, and $S(b')$ is the expected score of board state $b'$.

$F$ is a function that represents a strategy. For the optimal strategy, $F$ is the $max$ function. For the worst strategy, $F$ is the $min$ function. $F$ could also be a function that represents a human player's strategy, such as choosing the combination of tiles that contains the highest tile number.

Additionally in order to use this formula, the expected score of each board state $b$ where $B(i, b)$ is the empty set must be defined (i.e. what is the score of a board state when a roll causes the game to be over or all numbers are flipped). From the traditional rules of the game, the expected score of a board state when the game is over is the sum of all tiles that were flipped down. Other scoring systems will be explored further below.

> Note: Because each board state is always increasing the sum of all tiles, the formula above does not need to consider infinite loops.



### Example Calculation

Below is an example calculation of the expected score of the board state where the only remaining tiles are 2 and 1.

| Roll | Probability | Resulting Board States (Remaining Unflipped) | Expected Score |
| ---- | ----------- | ---------------------- | -------------- |
| 2    | 1/36        | 1                      | 44             |
| 3    | 2/36        | -                      | 45             |
| everything else | 33/36 | 1 2               | 42             |

$S(b) = \frac{1}{36} \cdot 44 + \frac{2}{36} \cdot 45 + \frac{33}{36} \cdot 42 = 42.222$


### Overall Expected Score and Optimal Strategy

The overall expected score of the game is the expected score, $S(b)$, of the initial board state.
The optimal strategy is the strategy, $F$, that returns the maximum expected score.

## Code for Solving

### Dependencies

Imports, constants, and dependency code for displaying results

In [ ]:
import itertools
from typing import Callable, Iterable
from IPython.display import display, Markdown

DEFAULT_DICE_COUNT = 2
DEFAULT_DICE_SIZE = 6
DEFAULT_MAX_DIGIT = 9

Table = list[dict[str, str|int|float]]
"""table of data"""

def display_table(table: Table) -> None:
    """display table of data"""
    if len(table) == 0:
        return
    keys = table[0].keys()
    output = "| " + " | ".join(keys) + " |\n"
    output += "| " + " | ".join(["---"] * len(keys)) + " |\n"
    for row in table:
        output += "| " + " | ".join([str(row[key]) for key in keys]) + " |\n"
    display(Markdown(output))

### Classes for Dice and Boards

In [ ]:
class Dice:
    """Utility class for calculating dice probabilities"""

    def __init__(self, dice_count: int = DEFAULT_DICE_COUNT, dice_size: int = DEFAULT_DICE_SIZE):
        assert isinstance(dice_count, int)
        assert 0 < dice_count < 8
        assert isinstance(dice_size, int)
        assert 1 < dice_size < 21
        dice_rolls = list(range(1, dice_size + 1))
        dice_outcomes = itertools.product(dice_rolls, repeat=dice_count)
        self._probabilities = {}
        for outcome in dice_outcomes:
            outcome_sum = sum(outcome)
            self._probabilities[outcome_sum] = self._probabilities.get(outcome_sum, 0) + 1
        for roll in self._probabilities:
            self._probabilities[roll] /= dice_size ** dice_count

    def all_rolls(self) -> Iterable[int]:
        """generates all possible rolls"""
        return self._probabilities.keys()

    def get_probability(self, roll: int) -> float:
        """returns the probability of rolling the given roll"""
        return self._probabilities.get(roll, 0)


class Board:
    """represents an immutable board state with a unique id"""

    def __init__(self, board_id: int, max_digit: int = DEFAULT_MAX_DIGIT):
        assert isinstance(max_digit, int)
        assert 0 < max_digit < 16 # 16 avoids long calculations
        assert isinstance(board_id, int)
        assert 0 <= board_id < 2 ** max_digit
        self._id = (max_digit, board_id) # tuple for immutability and hashing
        # calculate the board using binary representation of board_id
        binary = bin(board_id)[2:].zfill(max_digit)
        self._flipped_tiles = {i+1: char == '1' for i, char in enumerate(binary)}
        self._sum = None # lazy evaluation

    # required for use in sets and dicts
    def __hash__(self) -> int:
        """hashes the board id"""
        return hash(self._id)

    # required for use in sets and dicts
    def __eq__(self, other) -> bool:
        """compares two boards by id"""
        return self.max_digit() == other.max_digit() and self.get_id() == other.get_id()

    def __str__(self) -> str:
        """returns a string representation of the board"""
        return ' '.join('-' if is_flipped else str(tile) for tile, is_flipped in self._flipped_tiles.items())

    def __repr__(self) -> str:
        """returns Board(unique_id, sum, flipped_digits)"""
        return f'Board({self._id[1]}, {self.get_sum()}, {str(self)})'

    def max_digit(self) -> int:
        """returns the max digit of the board"""
        return self._id[0]

    def get_id(self) -> int:
        """returns the unique id of the board"""
        return self._id[1]

    def max_board_id(self) -> int:
        """returns the max possible score of the board"""
        return 2 ** self.max_digit() - 1

    def is_flipped(self, digit: int) -> bool:
        """returns True if the tile is flipped"""
        return self._flipped_tiles.get(digit, False)

    def get_sum(self) -> int:
        """returns the sum of the flipped digits"""
        if self._sum is None:
            self._sum = sum(tile for tile, is_flipped in self._flipped_tiles.items() if is_flipped)
        return self._sum

    def can_make_next_board(self, next_board: 'Board') -> bool:
        """returns True if the board can make a next board,
        the next board cannot have unflipped digits that the current board does have flipped"""
        assert isinstance(next_board, Board)
        assert self.max_digit() == next_board.max_digit()
        flipped_tiles = (tile for tile, is_flipped in self._flipped_tiles.items() if is_flipped)
        return all(next_board.is_flipped(tile) for tile in flipped_tiles)

    def is_all_flipped(self) -> bool:
        """returns True if all digits are flipped"""
        return self.get_id() == self.max_board_id()

    def get_flipped_count(self) -> int:
        """returns the number of flipped tiles"""
        return bin(self.get_id()).count('1')


### Type Definitions for Strategies and Scoring Systems

In [ ]:
BoardSet = Iterable[tuple[Board, float]]
"""a set of boards with their expected scores"""

BoardSums = dict[int, set[Board]]
"""a dictionary of sums and the boards that have that sum"""

Strategy = Callable[[BoardSet], Board]
"""a function that takes a set of boards and returns a chosen board"""

ScoringSystem = Callable[[Board], int]
"""a function that takes a board and returns a score, it is assumed that the board given is game over"""

ExpectedScores = dict[Board, float]
"""a dictionary of boards and their expected scores"""

pass # avoids doctype output for this cell

### Utility Class for Managing Boards

In [ ]:
class BoardManager:
    """manages all boards for a given max digit"""

    def __init__(self, max_digit: int = DEFAULT_MAX_DIGIT):
        assert isinstance(max_digit, int)
        assert 0 < max_digit < 16 # 16 avoids long calculations
        self._max_digit = max_digit
        self._max_board_sum = max_digit * (max_digit + 1) // 2
        self._boards = [Board(board_id, max_digit) for board_id in range(2 ** max_digit)]
        self._boards_by_sum = dict()
        for board in self._boards:
            board_sum = board.get_sum()
            if board_sum not in self._boards_by_sum:
                self._boards_by_sum[board_sum] = set()
            self._boards_by_sum[board_sum].add(board)

    def get_max_board_sum(self) -> int:
        """returns the max board sum"""
        return self._max_board_sum

    def get_boards(self) -> Iterable[Board]:
        """returns all boards"""
        return (board for board in self._boards)

    def get_boards_by_sum(self) -> BoardSums:
        """returns a dictionary of boards by sum"""
        # deep copy to avoid mutation
        dict_copy = dict()
        for key, value in self._boards_by_sum.items():
            dict_copy[key] = value.copy()
        return dict_copy

    def get_empty_expected_scores(self) -> ExpectedScores:
        """returns an empty dictionary of boards and their expected scores"""
        return {board: 0.0 for board in self._boards}

### Code for Calculating Expected Scores

In [ ]:
def find_possible_moves(boards_by_sum: dict[int, set[Board]], current_board: Board, roll: int) -> set[Board]:
    """helper for finding possible moves"""
    next_board_sum = current_board.get_sum() + roll
    return {board for board in boards_by_sum.get(next_board_sum, set()) if current_board.can_make_next_board(board)}


def create_board_set(possible_boards: set[Board], expected_scores: ExpectedScores) -> BoardSet:
    """helper for creating a board set"""
    return ((board, expected_scores[board]) for board in possible_boards)


def calculate_expected_scores(
        scoring_system: ScoringSystem,
        strategy: Strategy,
        board_manager: BoardManager,
        dice: Dice,
) -> ExpectedScores:
    """returns a dictionary of boards and their expected scores"""
    # initialize boards_by_sum and expected_scores
    boards_by_sum = board_manager.get_boards_by_sum()
    expected_scores = board_manager.get_empty_expected_scores()
    # calculate expected scores in reverse order of board sum
    for score in range(board_manager.get_max_board_sum(), -1, -1):
        for board in boards_by_sum.get(score, set()):
            cumulative_expected = 0.0
            for roll in dice.all_rolls():
                possible_boards = find_possible_moves(boards_by_sum, board, roll)
                if not possible_boards:
                    multiplier = scoring_system(board)
                else:
                    board_set = create_board_set(possible_boards, expected_scores)
                    multiplier = expected_scores[strategy(board_set)]
                cumulative_expected += dice.get_probability(roll) * multiplier
            expected_scores[board] = cumulative_expected
    return expected_scores

## Strategies

Below are the three strategies that were tested.

**`strategy_max`**: The optimal strategy. Chooses the board state with the highest expected score. This is the mathematically optimal strategy.

**`strategy_min`**: The worst strategy. Chooses the board state with the lowest expected score. This is the mathematically worst strategy.

**`strategy_highest_tile`**: A strategy that chooses the board state that flips the least amount fo tiles and flips the highest tile number possible. This is a strategy that a human player might use and what is hypothesized to be the optimal strategy.

In [ ]:
def strategy_max(board_set: BoardSet) -> Board:
    """returns the board with the highest expected score"""
    return max(board_set, key=lambda board_and_score: board_and_score[1])[0]


def strategy_min(board_set: BoardSet) -> Board:
    """returns the board with the lowest expected score"""
    return min(board_set, key=lambda board_and_score: board_and_score[1])[0]


def strategy_highest_tile(board_set: BoardSet) -> Board:
    """returns the board with the highest tile number and least amount of flipped tiles"""
    def flipped_tiles(board: Board) -> str:
        """helper for sorting by flipped tiles"""
        tile_string = ''
        for tile in range(board.max_digit(), 0, -1):
            tile_string += str(tile) if board.is_flipped(tile) else '0'
        return int(tile_string)
    boards = [board_and_score[0] for board_and_score in board_set]
    return max(boards, key=lambda board: (-board.get_flipped_count(), flipped_tiles(board)))

## Scoring Systems

Three different scoring systems are defined below:

**`score_traditional`**: The traditional scoring system. The score is the sum of all tiles that were flipped down.

**`score_all_or_nothing`**: The score is 0 if the game is lost and 1 if all tiles are flipped down. The traditional scoring system may produce a local maximum where certain moves are picked to increase the overall sum but make it more likely to lose the game. This scoring system is designed to avoid that local maximum. Hypothetical example: if the 3 tile is extremely hard to flip, it may be better to flip it down early when rolling a 9 instead of flipping down the 8 and 1, but flipping down the 8 and 1 would increase the overall expected sum.

**`score_flipped_count`**: The score is the count of tiles that were flipped down. This is my own added scoring system. If the strategy for the traditional scoring system is both simple and optimal, then the game is less interesting. This scoring system may produce a more interesting game that introduces more nuance, risk and reward.

In [ ]:
def score_traditional(board: Board) -> int:
    """returns the traditional score of a board"""
    return board.get_sum()


def score_all_or_nothing(board: Board) -> int:
    """returns 1 if all tiles are flipped, otherwise 0"""
    return 1 if board.is_all_flipped() else 0


def score_flipped_count(board: Board) -> int:
    """returns the all or nothing score of a board"""
    return board.get_flipped_count()

## Results

Calculating the expected scores for each strategy and scoring system combination

In [ ]:
def get_optimal_move(current_board: Board, possible_boards: set[Board], expected_scores: ExpectedScores, strategy: Strategy) -> str:
    """returns the optimal move(s) for a given board
    as the string representation of list[list]
    where each tuple is the digits flipped on the board

    if the expected score is within 0.0001 of the max expected score, then the move is considered optimal"""
    board_set = create_board_set(possible_boards, expected_scores)
    optimal_moves = list()
    optimal_expected_score = expected_scores[strategy(board_set)]
    for board in possible_boards:
        if abs(expected_scores[board] - optimal_expected_score) <= 0.0001:
            flipped_tiles = [tile for tile, is_flipped in board._flipped_tiles.items() if is_flipped and not current_board.is_flipped(tile)]
            flipped_tiles.sort()
            optimal_moves.append((flipped_tiles))
    optimal_moves.sort(key=lambda move: tuple(move))
    return str(optimal_moves)[1:-1]


def calculate_results() -> dict:
    """calculates the results for all scoring systems and strategies"""
    board_manager = BoardManager()
    dice = Dice()
    # run all combinations of scoring systems and strategies
    es_traditional_max = calculate_expected_scores(score_traditional, strategy_max, board_manager, dice)
    es_traditional_min = calculate_expected_scores(score_traditional, strategy_min, board_manager, dice)
    es_traditional_highest_tile = calculate_expected_scores(score_traditional, strategy_highest_tile, board_manager, dice)
    es_all_or_nothing_max = calculate_expected_scores(score_all_or_nothing, strategy_max, board_manager, dice)
    es_all_or_nothing_min = calculate_expected_scores(score_all_or_nothing, strategy_min, board_manager, dice)
    es_all_or_nothing_highest_tile = calculate_expected_scores(score_all_or_nothing, strategy_highest_tile, board_manager, dice)
    es_flipped_count_max = calculate_expected_scores(score_flipped_count, strategy_max, board_manager, dice)
    es_flipped_count_min = calculate_expected_scores(score_flipped_count, strategy_min, board_manager, dice)
    es_flipped_count_highest_tile = calculate_expected_scores(score_flipped_count, strategy_highest_tile, board_manager, dice)
    # create tabular data for expected scores by board state
    traditional_board_table: Table = []
    all_or_nothing_board_table: Table = []
    flipped_count_board_table: Table = []
    for board in board_manager.get_boards():
        traditional_board_table.append({
            'Board Id': board.get_id(),
            'Board': str(board),
            'Sum': board.get_sum(),
            'Max ES': es_traditional_max[board],
            'Min ES': es_traditional_min[board],
            'Highest Tile ES': es_traditional_highest_tile[board],
        })
        all_or_nothing_board_table.append({
            'Board Id': board.get_id(),
            'Board': str(board),
            'Sum': board.get_sum(),
            'Max ES': es_all_or_nothing_max[board],
            'Min ES': es_all_or_nothing_min[board],
            'Highest Tile ES': es_all_or_nothing_highest_tile[board],
        })
        flipped_count_board_table.append({
            'Board Id': board.get_id(),
            'Board': str(board),
            'Sum': board.get_sum(),
            'Max ES': es_flipped_count_max[board],
            'Min ES': es_flipped_count_min[board],
            'Highest Tile ES': es_flipped_count_highest_tile[board],
        })
    # create tabular data for optimal moves
    traditional_move_table: Table = []
    all_or_nothing_move_table: Table = []
    flipped_count_move_table: Table = []
    for board in board_manager.get_boards():
        for roll in dice.all_rolls():
            possible_boards = find_possible_moves(board_manager.get_boards_by_sum(), board, roll)
            # skip if no possible moves
            if not possible_boards:
                continue
            traditional_move_table.append({
                'Board Id': board.get_id(),
                'Board': str(board),
                'Sum': board.get_sum(),
                'Roll': roll,
                'Max Move': get_optimal_move(board, possible_boards, es_traditional_max, strategy_max),
                'Min Move': get_optimal_move(board, possible_boards, es_traditional_min, strategy_min),
                'Highest Tile Move': get_optimal_move(board, possible_boards, es_traditional_highest_tile, strategy_highest_tile),
            })
            all_or_nothing_move_table.append({
                'Board Id': board.get_id(),
                'Board': str(board),
                'Sum': board.get_sum(),
                'Roll': roll,
                'Max Move': get_optimal_move(board, possible_boards, es_all_or_nothing_max, strategy_max),
                'Min Move': get_optimal_move(board, possible_boards, es_all_or_nothing_min, strategy_min),
                'Highest Tile Move': get_optimal_move(board, possible_boards, es_all_or_nothing_highest_tile, strategy_highest_tile),
            })
            flipped_count_move_table.append({
                'Board Id': board.get_id(),
                'Board': str(board),
                'Sum': board.get_sum(),
                'Roll': roll,
                'Max Move': get_optimal_move(board, possible_boards, es_flipped_count_max, strategy_max),
                'Min Move': get_optimal_move(board, possible_boards, es_flipped_count_min, strategy_min),
                'Highest Tile Move': get_optimal_move(board, possible_boards, es_flipped_count_highest_tile, strategy_highest_tile),
            })
    return {
        'board_manager': board_manager,
        'dice': dice,
        'traditional_board_table': traditional_board_table,
        'all_or_nothing_board_table': all_or_nothing_board_table,
        'flipped_count_board_table': flipped_count_board_table,
        'traditional_move_table': traditional_move_table,
        'all_or_nothing_move_table': all_or_nothing_move_table,
        'flipped_count_move_table': flipped_count_move_table,
        'es_traditional_max': es_traditional_max,
        'es_traditional_min': es_traditional_min,
        'es_traditional_highest_tile': es_traditional_highest_tile,
        'es_all_or_nothing_max': es_all_or_nothing_max,
        'es_all_or_nothing_min': es_all_or_nothing_min,
        'es_all_or_nothing_highest_tile': es_all_or_nothing_highest_tile,
        'es_flipped_count_max': es_flipped_count_max,
        'es_flipped_count_min': es_flipped_count_min,
        'es_flipped_count_highest_tile': es_flipped_count_highest_tile
    }

full_results = calculate_results()

## Results for Traditional Scoring System

> See **Exhibit D** for the optimal moves for the traditional scoring system.

After looking at the results of the optimal moves for the traditional scoring system, flipping down the highest possible tiles each turn is clear pattern. It is not the best move 100% of the time. However, when it is not the best move, the expected score is still very close to the optimal expected score. This is thebest fitting pattern that a human can use without having to memorize the optimal moves for every possible board state. Below is table of exceptions to the pattern.

> See **Exhibit G** for the full table comparing the optimal moves to the highest tile moves.

In [ ]:
def results_traditional_scoring_system(results: dict) -> Table:
    """a comparison of the optimal strategy and the highest tile strategy for the traditional scoring system"""
    exhibit_table = []
    results_subtable = results['traditional_move_table']
    es_max = results['es_traditional_max']
    es_highest_tile = results['es_traditional_highest_tile']
    for record in results_subtable:
        if record['Max Move'] == record['Highest Tile Move']:
            continue
        board = Board(record['Board Id'])
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Max ES': f'{es_max[board]:.3f}',
            'Highest Tile Move': record['Highest Tile Move'],
            'Highest Tile ES': f'{es_highest_tile[board]:.3f}'
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


# display_table(results_traditional_scoring_system(full_results))

## Appendix

### Exhibit A: All Possible Board States with Expected Score of Each Strategy for Traditional Scoring System

In [ ]:
def exhibit_a(results: dict) -> Table:
    """returns a table for expected scores of the traditional scoring system"""
    exhibit_table = []
    results_subtable = results['traditional_board_table']
    for record in results_subtable:
        strategic_difference = record['Max ES'] - record['Min ES']
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Max ES': f'{record["Max ES"]:.3f}',
            'Min ES': f'{record["Min ES"]:.3f}',
            'Highest Tile ES': f'{record["Highest Tile ES"]:.3f}',
            'Strategic Difference': f'{strategic_difference:.3f}',
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: record['Board Sum'])
    return exhibit_table

# display_table(exhibit_a(full_results))

### Exhibit B: All Possible Board States with Expected Score of Each Strategy for All or Nothing Scoring System

In [ ]:
def exhibit_b(results: dict) -> Table:
    """returns a table for expected scores of the all or nothing scoring system"""
    exhibit_table = []
    results_subtable = results['all_or_nothing_board_table']
    for record in results_subtable:
        strategic_difference = record['Max ES'] - record['Min ES']
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Max ES': f'{record["Max ES"]:.3f}',
            'Min ES': f'{record["Min ES"]:.3f}',
            'Highest Tile ES': f'{record["Highest Tile ES"]:.3f}',
            'Strategic Difference': f'{strategic_difference:.3f}',
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: record['Board Sum'])
    return exhibit_table

# display_table(exhibit_b(full_results))

### Exhibit C: All Possible Board States with Expected Score of Each Strategy for Flipped Count Scoring System

In [ ]:
def exhibit_c(results: dict) -> Table:
    """returns a table for expected scores of the flipped count scoring system"""
    exhibit_table = []
    results_subtable = results['flipped_count_board_table']
    for record in results_subtable:
        strategic_difference = record['Max ES'] - record['Min ES']
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Max ES': f'{record["Max ES"]:.3f}',
            'Min ES': f'{record["Min ES"]:.3f}',
            'Highest Tile ES': f'{record["Highest Tile ES"]:.3f}',
            'Strategic Difference': f'{strategic_difference:.3f}',
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: record['Board Sum'])
    return exhibit_table

# display_table(exhibit_c(full_results))

### Exhibit D: Optimal Moves for Traditional Scoring System

Note: Only showing results where there is a difference between the optimal strategy and the worst strategy

In [ ]:
def exhibit_d(results: dict) -> Table:
    """returns a table for optimal moves of the traditional scoring system"""
    exhibit_table = []
    results_subtable = results['traditional_move_table']
    for record in results_subtable:
        if record['Max Move'] == record['Min Move']:
            continue
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Min Move': record['Min Move'],
            'Highest Tile Move': record['Highest Tile Move'],
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


# display_table(exhibit_d(full_results))

### Exhibit E: Optimal Moves for All or Nothing Scoring System

Note: Only showing results where there is a difference between the optimal strategy and the worst strategy

In [ ]:
def exhibit_e(results: dict) -> Table:
    """returns a table for optimal moves of the all or nothing scoring system"""
    exhibit_table = []
    results_subtable = results['all_or_nothing_move_table']
    for record in results_subtable:
        if record['Max Move'] == record['Min Move']:
            continue
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Min Move': record['Min Move'],
            'Highest Tile Move': record['Highest Tile Move'],
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


# display_table(exhibit_e(full_results))

### Exhibit F: Optimal Moves for Flipped Scoring System

Note: Only showing results where there is a difference between the optimal strategy and the worst strategy

In [ ]:
def exhibit_f(results: dict) -> Table:
    """returns a table for optimal moves of the flipped count scoring system"""
    exhibit_table = []
    results_subtable = results['flipped_count_move_table']
    for record in results_subtable:
        if record['Max Move'] == record['Min Move']:
            continue
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Min Move': record['Min Move'],
            'Highest Tile Move': record['Highest Tile Move'],
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


# display_table(exhibit_f(full_results))

### Exhibit G: Full Comparison of Moves for Optimal Strategy and Highest Tile Strategy

Note: traditional scoring system is used for this comparison

In [ ]:
def exhibit_g(results: dict) -> Table:
    """a comparison of the optimal strategy and the highest tile strategy for the traditional scoring system"""
    exhibit_table = []
    results_subtable = results['traditional_move_table']
    es_max = results['es_traditional_max']
    es_highest_tile = results['es_traditional_highest_tile']
    for record in results_subtable:
        board = Board(record['Board Id'])
        exhibit_table.append({
            'Board (Remaining)': f'`{record["Board"]}`',
            'Board Sum': record['Sum'],
            'Roll': record['Roll'],
            'Max Move': record['Max Move'],
            'Max ES': f'{es_max[board]:.3f}',
            'Highest Tile Move': record['Highest Tile Move'],
            'Highest Tile ES': f'{es_highest_tile[board]:.3f}'
        })
    # sort by board sum ascending
    exhibit_table.sort(key=lambda record: (record['Board Sum'], record['Board (Remaining)'], record['Roll']))
    return exhibit_table


# display_table(exhibit_g(full_results))